# Hackathon Scenario 2: Multiple Vaccines and Variants
Models from early in the Covid-19 pandemic were relatively simple, as we had no vaccines available, only NPIs to mitigate the spread of the disease, and only one strain (the original wild type) to worry about. By mid-2021, the situation was much more complex, as we had access to multiple vaccines, each with multiple doses, and the emergence of multiple variants. You are interested in modeling this situation with SV2(AIR)3 (https://www.nature.com/articles/s41598- 022-06159-x, https://doi.org/10.1038/s41598-022-06159-x), one of the first models to represent multiple Covid-19 variants, and the competition between them.

1. Extract the simplest version of the model with 1 vaccine and 1 variant (shown in Fig. 1A)

  a. (TA1 Model Extraction Workflow) Extract the model from Matlab code
‘svair_simple.m’ and ‘get_beta.m’ and with the publication equations and Fig. 1A (with the exception of one small difference between code and figure). Note that in this code, the units of all state variables are in terms of fraction of the total population.
  
  b. (TA3 Simulation Workflow) Simulate the first 14 months of the pandemic beginning on January 1st, 2020. Consider only the Wild Type variant, and no vaccination during this period. Use parameter values from the supplementary material, for the wild type. Use the population of Ontario in 2019: 14.57 million people, and let the natural death rate 𝜇 = 2.05𝑒-5. For initial conditions, let 𝐼(0) = 1e-6, 𝑆(0) = 1 − 𝑠𝑢𝑚(𝐼(0) + 𝐴(0)), and all other values be 0. When do I(t) and A(t) peak, and what is the value of these variables at their peaks? How do the I(t) and A(t) profiles compare with Fig. 3d,f (which considered 3 variants and 2 types of vaccines)? Given that this question assumes only the presence of the Wild Type variant, do the results seem reasonable?
  
  c. (TA2 Domain Knowledge Grounding; ASKEM Workbench Only) Demonstrate that the domain knowledge groundings accurately reflect new concepts with the extensions such as “variants”.

In [ ]:
import sympy
from copy import deepcopy as _d
from mira.metamodel import *
from mira.modeling import Model
from mira.modeling.amr.petrinet import AMRPetriNetModel
from mira.modeling.viz import GraphicalModel

## Represent original beta_scale timeline from the model

In [ ]:
times = [0, 82, 198, 282, 327, 370, 429, 468, 527, 548, 562, 609, 670, 731, 1096]
beta_scales = [1, 0.4, 0.55, 0.6, 0.55, 0.3, 0.4, 0.15, 0.2, 0.3, 0.35, 0.4, 0.5, 0.4]

t = sympy.Symbol('t')

beta_scale_var = sympy.Piecewise(
    *[(b, (tl <= t) & (t < tu))
      for (tl, tu), b in zip(zip(times[:-1], times[1:]), beta_scales)],
    (1, True)
)

In [ ]:
beta_scale_var

## Alternative: model a static beta_scale and handle timeline during simulation

In [ ]:
beta_scale_static = 1.0

#### Select between two beta_scales

In [ ]:
beta_scale_type = 'static'  # var
if beta_scale_type == 'static':
    beta_scale = beta_scale_static
else:
    beta_scale = beta_scale_var

In [ ]:
person_units = Unit(expression=sympy.Symbol('person'))
day_units = Unit(expression=sympy.Symbol('day'))
per_day_units = Unit(expression=1/sympy.Symbol('day'))
dimensionless_units = Unit(expression=sympy.Integer('1'))

populations = {
    # Susceptible
    'S': [{'ido': '0000514'}, {'vaccination_status': 'vo:0001377'}],
    # Susceptible lost immunity
    'SVR': [{'ido': '0000514'}, {'disease_history': 'ido:0000592'}],
    # Vaccinated once
    'V1': [{'ido': '0000514'}, {'vaccination_status': 'askemo:0000018'}],
    # Vaccinated twice
    'V2': [{'ido': '0000514'}, {'vaccination_status': 'askemo:0000019'}],
    # Infected
    'I': [{'ido': '0000511'}, {}],
    # Infected vaccinated
    'IV': [{'ido': '0000511'}, {'vaccination_status': 'vo:0001376'}],
    # Infected after recovery
    'IR': [{'ido': '0000511'}, {'disease_history': 'ido:0000592'}],
    # Asymptomatic
    'A': [{'ido': '0000511'}, {'disease_severity': 'ncit:C3833'}],
    # Asymptomatic after recovery
    'AR': [{'ido': '0000511'}, {'disease_severity': 'ncit:C3833'}],
    # Recovered
    'R': [{'ido': '0000514'}, {'disease_severity': 'ido:0000592'}],
    # Recovered after second infection
    'R2': [{'ido': '0000592'}, {}],
}

c = {
    pop: Concept(name=pop, units=dimensionless_units,
                 identifiers=indentifiers, context=context)
    for pop, (indentifiers, context) in populations.items()
}

N = 14570000

parameters = [
    Parameter(name='beta', value=3.3e-9*N, units=per_day_units),
    Parameter(name='beta_v1', value=0.2*3.3e-9*N, units=per_day_units),
    Parameter(name='beta_v2', value=0.05*3.3e-9*N, units=per_day_units),
    Parameter(name='beta_R', value=0.05*3.3e-9*N, units=per_day_units),
    Parameter(name='gamma', value=1/28, units=per_day_units),
    Parameter(name='mu', value=109019/(14570000*365), units=per_day_units),
    Parameter(name='mu_I', value=0.001, units=per_day_units),
    Parameter(name='mu_IV', value=0.15*0.001, units=per_day_units),
    Parameter(name='nu_R', value=1/(4*365), units=per_day_units),
    Parameter(name='nu_v1', value=1/365, units=per_day_units),
    Parameter(name='nu_v2', value=1/(4*365), units=per_day_units),
    Parameter(name='ai', value=0.5, units=dimensionless_units), # paper uses sigma instead of ai
    Parameter(name='ai_V', value=0.85, units=dimensionless_units),
    Parameter(name='ai_R', value=0.85, units=dimensionless_units),
    Parameter(name='ai_beta_ratio', value=3.0, units=dimensionless_units),
]

if beta_scale_type == 'static':
    parameters.append(Parameter(name='beta_scale', value=1.0, units=dimensionless_units))
    beta_scale = sympy.Symbol('beta_scale')

initials = {
    'S': Initial(concept=c['S'], expression=1-1e-6),
    'SVR': Initial(concept=c['SVR'], expression=0.0),
    'V1': Initial(concept=c['V1'], expression=0.0),
    'V2': Initial(concept=c['V2'], expression=0.0),
    'I': Initial(concept=c['I'], expression=1e-6),
    'IV': Initial(concept=c['IV'], expression=0.0),
    'IR': Initial(concept=c['IR'], expression=0.0),
    'A': Initial(concept=c['A'], expression=0.0),
    'AR': Initial(concept=c['AR'], expression=0.0),
    'R': Initial(concept=c['R'], expression=0.0),
    'R2': Initial(concept=c['R2'], expression=0.0),
}

In [ ]:
S, SVR, V1, V2, I, IV, IR, A, AR, R, R2 = sympy.symbols('S SVR V1 V2 I IV IR A AR R R2')

In [ ]:
beta, beta_v1, beta_v2, beta_R, gamma, mu, mu_I, mu_IV, nu_R, nu_v1, nu_v2, ai, ai_V, ai_R, ai_beta_ratio = \
    sympy.symbols('beta beta_v1 beta_v2 beta_R gamma mu mu_I mu_IV nu_R nu_v1 nu_v2 ai ai_V ai_R ai_beta_ratio')

In [ ]:
all_infectious = [c['I'], c['IV'], c['IR'], c['A'], c['AR']]
inf_terms = [I, IV, IR, ai_beta_ratio*A, ai_beta_ratio*AR]

templates = []
for inf, inf_term in zip(all_infectious, inf_terms):
    # Infection asym
    t = ControlledConversion(subject=c['S'], outcome=c['A'],
                             controller=_d(inf),
                             rate_law=ai*beta*beta_scale*S*inf_term)
    templates.append(t)
    t = ControlledConversion(subject=c['V1'], outcome=c['A'],
                             controller=_d(inf),
                             rate_law=ai*beta*beta_scale*V1*inf_term)
    templates.append(t)
    t = ControlledConversion(subject=c['V2'], outcome=c['A'],
                             controller=_d(inf),
                             rate_law=ai*beta*beta_scale*V2*inf_term)
    templates.append(t)

    # Infection sym
    t = ControlledConversion(subject=c['S'], outcome=c['I'],
                             controller=_d(inf),
                             rate_law=(1-ai)*beta*beta_scale*S*inf_term)
    templates.append(t)
    t = ControlledConversion(subject=c['V1'], outcome=c['IV'],
                             controller=_d(inf),
                             rate_law=ai*beta_v1*beta_scale*V1*inf_term)
    templates.append(t)
    t = ControlledConversion(subject=c['V2'], outcome=c['IV'],
                             controller=_d(inf),
                             rate_law=ai*beta_v2*beta_scale*V2*inf_term)
    templates.append(t)
    
    
    # Reinfection asym
    t = ControlledConversion(subject=c['R'], outcome=c['AR'],
                             controller=_d(inf),
                             rate_law=ai_R*beta_R*beta_scale*R*inf_term)
    templates.append(t)
    # Reinfection sym
    t = ControlledConversion(subject=c['R'], outcome=c['IR'],
                             controller=_d(inf),
                             rate_law=(1-ai_R)*beta_R*beta_scale*R*inf_term)
    templates.append(t)

templates += [
    # Vaccination
    # Note: structurally we want to have these in the model but we make the rates 0
    NaturalConversion(subject=c['S'], outcome=c['V1'], rate_law=0*S),
    NaturalConversion(subject=c['A'], outcome=c['V1'], rate_law=0*A),
    NaturalConversion(subject=c['V1'], outcome=c['V2'], rate_law=0*V1),

    # Recovery
    NaturalConversion(subject=c['I'], outcome=c['R'], rate_law=gamma*I),
    NaturalConversion(subject=c['IV'], outcome=c['R'], rate_law=gamma*IV),
    NaturalConversion(subject=c['A'], outcome=c['R'], rate_law=gamma*A),

    # Second recovery
    NaturalConversion(subject=c['IR'], outcome=c['R2'], rate_law=gamma*IR),
    NaturalConversion(subject=c['AR'], outcome=c['R2'], rate_law=gamma*AR),

    # Death
    NaturalDegradation(subject=c['S'], rate_law=mu*S),
    NaturalDegradation(subject=c['SVR'], rate_law=mu*SVR),
    NaturalDegradation(subject=c['V1'], rate_law=mu*V1),
    NaturalDegradation(subject=c['V2'], rate_law=mu*V2),
    NaturalDegradation(subject=c['I'], rate_law=mu_I*I),
    NaturalDegradation(subject=c['IV'], rate_law=mu_IV*IV),
    NaturalDegradation(subject=c['IR'], rate_law=mu*IR),
    NaturalDegradation(subject=c['A'], rate_law=mu*A),
    NaturalDegradation(subject=c['AR'], rate_law=mu*AR),
    NaturalDegradation(subject=c['R'], rate_law=mu*R),
    NaturalDegradation(subject=c['R2'], rate_law=mu*R2),
    
    # Birth
    NaturalProduction(outcome=c['S'], rate_law=mu),

    # Loss of immunity
    NaturalConversion(subject=c['V1'], outcome=c['SVR'], rate_law=nu_v1*V1),
    NaturalConversion(subject=c['V2'], outcome=c['SVR'], rate_law=nu_v2*V2),
    NaturalConversion(subject=c['R'], outcome=c['SVR'], rate_law=nu_R*R),
    NaturalConversion(subject=c['R2'], outcome=c['SVR'], rate_law=nu_R*R2),
]

In [ ]:
tm = TemplateModel(templates=templates,
                   parameters={p.name: p for p in parameters},
                   initials=initials)
tm.annotations = Annotations(name="Scenario 2a")

%debug
AMRPetriNetModel(Model(tm)).to_json_file('scenario2_a_beta_scale_%s.json' % beta_scale_type)

## Modifying the model

2.Extend the model to incorporate multiple strains and 2 vaccine types, according to publication Fig. 1B and equations #2-12.

b.	(TA2 Model Modification/Extension Workflow) Alternately, use TA2 model extension/transformation tools, to extend the simpler model extracted from Question 1.  How does this extended model compare with the extracted model from 2a?

### Stratify by vaccine types

In [ ]:
from mira.metamodel.ops import stratify
tm_multi_vax = stratify(
    tm,
    key='vaccine_type',
    strata=['vo:0004987', 'vo:0005158'],
    structure=[],
    cartesian_control=True,
    concepts_to_stratify={'V1', 'V2', 'IV'},
    params_to_stratify={'beta_v1', 'beta_v2', 'ai_v', 'nu_v1', 'nu_v2', 'mu_IV'},
    )

In [ ]:
for t in tm_multi_vax.templates:
    if isinstance(t, NaturalProduction):
        print(t)

In [ ]:
tm_multi_vax.annotations.name = 'Scenario 2.2.b multi-vax'
tm_multi_vax.annotations.description = 'Scenario 2.2.b model with two different vaccine types '\
    '(Pfizer and AstraZeneca)'
AMRPetriNetModel(Model(tm_multi_vax)).to_json_file('scenario2_2_b_multi_vax.json')
GraphicalModel.for_jupyter(tm_multi_vax)

### Stratify by viral strains

In [ ]:
tm_multi_strain = stratify(
    tm_multi_vax,
    key='strain',
    strata=['wt', 'cido:0000371', 'cido:0000376'],
    structure=[],
    cartesian_control=True,
    concepts_to_stratify={'IV_vo:0004987', 'IV_vo:0005158', 'I', 'IR', 'A', 'AR', 'R'},
    params_to_stratify={'beta', 'beta_R', 'gamma', 'mu_I', 'mu_IV_vo:0004987', 'mu_IV_vo:0005158'}
    )

In [ ]:
tm_multi_strain.annotations.name = 'Scenario 2.2.b multi-vax/multi-strain'
tm_multi_strain.annotations.description = 'Scenario 2.2.b model with two different vaccine types '\
    '(Pfizer and AstraZeneca) and three variants (WT, Alpha, Delta)'
AMRPetriNetModel(Model(tm_multi_strain)).to_json_file('scenario2_2_b_multi_vax_multi_strain.json')
GraphicalModel.for_jupyter(tm_multi_strain)

## Adapt the model to the delta/omicron era

4.	Omicron Extension. The research in the paper was done before the Omicron variant, but the authors ran predictions for a fictional variant Neos, designed to be deadlier than previous variants, through increased infectivity, vaccine escape, and fraction of asymptomatic infections. There are some similarities between this fictional variant, and Omicron. Assume you are in the fall of 2021, at a time when Delta was on the downswing, and Omicron was emerging.
a.	(TA2 Model Modification Workflow) Simplify the model structure to still consider multiple vaccines, but only the Delta and Omicron variants

In [ ]:
tm_multi_strain_2 = stratify(
    tm_multi_vax,
    key='strain',
    strata=['cido:0000376', 'cido:0000590'],
    structure=[],
    cartesian_control=True,
    concepts_to_stratify={'IV_vo:0004987', 'IV_vo:0005158', 'I', 'IR', 'A', 'AR', 'R'},
    params_to_stratify={'beta', 'beta_R', 'gamma', 'mu_I', 'mu_IV_vo:0004987', 'mu_IV_vo:0005158'}
    )

In [ ]:
tm_multi_strain_2.annotations.name = 'Scenario 2.4.a multi-vax/Delta-Omicron'
tm_multi_strain_2.annotations.description = 'Scenario 2.4.a model with two different vaccine types ' \
    '(Pfizer and AstraZeneca) and two variants (Delta, Omicron)'
AMRPetriNetModel(Model(tm_multi_strain_2)).to_json_file('scenario2_4_a_multi_vax_delta_omicron.json')
GraphicalModel.for_jupyter(tm_multi_strain_2)

### Stratify by age group

5.Change the setting to the United States

c.	Challenge (TA2 Model Modification Workflow) As noted in the publication, a major limitation of the model is the lack of age structure, as mortality rates and impacts of NPIs affect different age groups at varying levels. Stratify the model to match the number of age groups found in US CDC vaccination and dosage data: https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-Jurisdi/unsk-b7fc. Then rerun the simulation for the time period January 1st, 2020 – December 31st, 2021. How do outcomes differ by age group? For United States contact matrices, refer to https://doi.org/10.1371/journal.pcbi.1005697; https://doi.org/10.1371/journal.pcbi.1009098

In [ ]:
tm_age = stratify(
    tm_multi_strain_2,
    key='age',
    strata=['0_5', '5_12', '12_18', '18_65', '65'],
    structure=[],
    cartesian_control=True,
    params_to_stratify={p for p in tm_multi_strain_2.parameters if p.startswith('beta_')}
    )

In [ ]:
tm_age.annotations.name = 'Scenario 2.5.c multi-vax/Delta-Omicron/age'
tm_age.annotations.description = 'Scenario 2.5.c model with two different vaccine types (' \
    'Pfizer and AstraZeneca) and two variants (Delta, Omicron) and 5 age groups.'
AMRPetriNetModel(Model(tm_age)).to_json_file('scenario2_5_c_multi_vax_delta_omicron_age.json')

## Model statistics

In [ ]:
import pandas as pd
data = [[m.annotations.name, len(m.templates), len(m.parameters), len(Model(m).variables)]
        for m in [tm, tm_multi_vax, tm_multi_strain, tm_multi_strain_2, tm_age]]
pd.DataFrame(data, columns=["Model", "#transitions", "#params", "#states"])